In [1]:
!pip install pypapago

In [2]:
import os 
import sys 
import urllib.request
import re
import pandas as pd
reg = re.compile(r'[^ㄱ-ㅣ가-힣]+')
co = pd.read_csv('./data/KO_token.txt',encoding='utf-8') #조사 불용어

In [9]:
result_df1 = pd.read_csv('./data/NM_EN_token.csv',encoding = 'utf-8-sig')
result_df2 = pd.read_csv('./data/NM_KO_hannanum.csv',encoding='utf-8-sig') 

In [10]:
client_id = "U9UBbQtpSbWt6Mzn6WdC"
client_secret = "rHPIy0JgYq"

In [12]:
input_str = input()
ko_response_word = ''
en_response_word = ''
ko_word =input_str
en_word = input_str
if reg.match(input_str):
    print("영어 --> 한글")
    encText = urllib.parse.quote(input_str) 
    data = "source=en&target=ko&text=the " + encText
    url = "https://openapi.naver.com/v1/papago/n2mt"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        print(response_body)
        ko_response_word =response_body.decode('utf-8')
    else:
        print("Error Code:" + rescode)
else:
    print("한글 --> 영어")
    encText = urllib.parse.quote(input_str) 
    data = "source=ko&target=en&text=" + encText
    url = "https://openapi.naver.com/v1/papago/n2mt"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        print(response_body)
        en_response_word =response_body.decode('utf-8')
    else:
        print("Error Code:" + rescode)

가축
한글 --> 영어
b'{"message":{"@type":"response","@service":"naverservice.nmt.proxy","@version":"1.0.0","result":{"srcLangType":"ko","tarLangType":"en","translatedText":"domestic animals","engineType":"PRETRANS","pivot":null}}}'


In [37]:
if len(ko_response_word)!= 0:
    ko_word = ko_response_word[ko_response_word.index('translatedText')+17:ko_response_word.index('engineType')-3]
if len(en_response_word)!= 0:
    en_word = en_response_word[en_response_word.index('translatedText')+17:en_response_word.index('engineType')-3]

In [38]:
en_word =en_word.lower()
print(ko_word)
print(en_word)

가축
domestic animals


In [39]:
#조사 없애기
import nltk
nltk.download('punkt')

string_list=co['KO'].tolist()

clean_words = [] 
for word in nltk.tokenize.word_tokenize(ko_word):
    if word not in string_list: #불용어 제거
        clean_words.append(word)
print(clean_words)
ko_word = ''.join(clean_words)        
print(ko_word) 

['가축']
가축


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DeepLearning_3\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [33]:
#영어도 명사만 --> 안됨
# 가축이면 domestic animals 인데, animals만 가능,,,
# 그렇다고 splic하기도 좀 그런데,, 
from tensorflow.keras.preprocessing.text import text_to_word_sequence
word_tokens = text_to_word_sequence(en_word)
tokens_pos  = nltk.pos_tag(word_tokens)
en = ''
for word , tag in tokens_pos:
    if len(tag) > 1:
        if tag in ['NN','NNS']:
            print(word, tag)
            en += (word + ' ')
en_word = en

animals NNS


In [35]:
try:
    result_df1.drop(['Unnamed: 0'],axis = 1, inplace = True)
except :
    pass

In [41]:
search = result_df2
result_ko = search[search['NM_KO'].str.contains(ko_word, na=False)]
#for index in result_ko['NM_KO']:
  #print(index)
search1 = result_df1
result_en = pd.DataFrame(columns=['HSCODE','HSCODE_6','NM_EN'])
result_en = result_en.append( search1[search1['NM_EN'].str.contains(en_word, na=False)],ignore_index=True)

#검색 결과가 0이면 split해서 검색 해보자. 
# 여기서 문제는 유사도가 이상함, 
# 가축을 검색했는데, domestic검색하고, animals검색하는거니깐,,,
# 유사도 해야겠네. 
if len(result_en) == 0:
    for word in en_word.split():
        print(word)
        result_en = result_en.append( search1[search1['NM_EN'].str.contains(word, na=False)],ignore_index=True)
result_en

domestic
animals


,HSCODE,HSCODE_6,NM_EN
0,105111000,10511,purebred breeding animals chicks purebred bree...
1,106199000,10619,other cat pet domestic shorthair live guineapi...
2,207129000,20712,other cut pieces chicken cut pieces frozen fow...
3,207141010,20714,leg chicken drum sticks chicken whole leg bone...
4,407210000,40721,fowls species chicken eggs fresh eggs fresh eg...
...,...,...,...
155,9503003493,950300,toys representing animals swimming duck wsb
156,9503003495,950300,ceramics ceramic coin bank ceramic doll animal...
157,9503003497,950300,wood wooden genga masai doll wood toys represe...
158,9503003499,950300,other miniature wooden toys disney figure eedi...
